# RF Experiments with Standard 20% Split 

In [3]:
%load_ext autotime
%matplotlib inline

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.86 ms (started: 2023-07-06 11:41:52 +03:00)


In [4]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import re

time: 1.51 ms (started: 2023-07-06 11:41:53 +03:00)


In [5]:
# Reading the data
data =  pd.read_csv('Data/SimpleApproach/Imports/DataImports2016-2019GravityHSlevelComplexity.csv')
data.dropna(inplace=True)
data

,refYear,partnerCode,reporterCode,distw_harmonic,contig,gdp_d,gdp_o,gdpcap_d,gdpcap_o,pop_d,...,wto_d,wto_o,refMonth,partner2Code,cmdCode,primaryValue,2HScmdCode,4HScmdCode,pci,density
0,2016,4.0,12.0,5758.0,0.0,1.946902e+07,1.560796e+08,0.562,3.844,34656.033,...,1.0,0.0,5.0,0.0,842123.0,163.030,84.0,8421.0,1.908080,0.010296
1,2016,8.0,12.0,1511.0,0.0,1.192689e+07,1.560796e+08,4.147,3.844,2876.101,...,1.0,0.0,8.0,0.0,842123.0,47.290,84.0,8421.0,1.908080,0.010296
2,2016,8.0,12.0,1511.0,0.0,1.192689e+07,1.560796e+08,4.147,3.844,2876.101,...,1.0,0.0,10.0,0.0,842123.0,1941.390,84.0,8421.0,1.908080,0.010296
3,2016,32.0,12.0,10149.0,0.0,5.458662e+08,1.560796e+08,12.449,3.844,43847.431,...,1.0,0.0,1.0,0.0,842123.0,812.090,84.0,8421.0,1.908080,0.010296
4,2016,32.0,12.0,10149.0,0.0,5.458662e+08,1.560796e+08,12.449,3.844,43847.431,...,1.0,0.0,2.0,0.0,842123.0,4504.280,84.0,8421.0,1.908080,0.010296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90844603,2019,458.0,352.0,11271.0,0.0,3.652764e+08,2.485774e+07,11.433,68.941,31949.789,...,1.0,1.0,8.0,0.0,846721.0,3629.278,84.0,8467.0,2.672168,0.034830
90844604,2019,458.0,352.0,11271.0,0.0,3.652764e+08,2.485774e+07,11.433,68.941,31949.789,...,1.0,1.0,10.0,0.0,846721.0,11405.271,84.0,8467.0,2.672168,0.034830
90844605,2019,484.0,352.0,7308.0,0.0,1.269404e+09,2.485774e+07,9.950,68.941,127575.531,...,1.0,1.0,1.0,0.0,846721.0,966.607,84.0,8467.0,2.672168,0.034830
90844606,2019,484.0,352.0,7308.0,0.0,1.269404e+09,2.485774e+07,9.950,68.941,127575.531,...,1.0,1.0,2.0,0.0,846721.0,1010.410,84.0,8467.0,2.672168,0.034830


time: 13min 1s (started: 2023-07-06 11:41:55 +03:00)


In [6]:
# Define evaluation metrics functions for model performance assessment
def MPE(Y_actual,Y_Predicted):
    mape = np.mean((Y_actual - Y_Predicted)/Y_actual)*100
    return mape

def MAPE(Y_actual,Y_Predicted):
    mpe = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mpe

# Standardize currency values by dividing by 1,000,000 for better readability and consistency

data['primaryValue'] = data['primaryValue']/1000000
data['gdp_o'] = data['gdp_o']/1000000
data['gdp_d'] = data['gdp_d']/1000000
data['gdpcap_o'] = data['gdpcap_o']/1000000
data['gdpcap_d'] = data['gdpcap_d']/1000000
data['distw_harmonic'] = data['distw_harmonic']/1000

time: 2.45 s (started: 2023-07-06 11:54:56 +03:00)


## RandomForest modeling


In [7]:
# Separating the dataset into features (X) and the target variable (y)
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

# Splitting the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y


# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

# Evaluating the model's performance and assessing its accuracy
r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))
index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])
print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 12.4min remaining: 26.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 14.7min remaining:  2.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 14.9min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.5s remaining:   52.1s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.9s remaining:    3.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.9s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   17.9s remaining:  3.4min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   19.8s remaining:   13.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   22.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.91
R2 on validation data: 0.88
Mean Squared Error: 8.94
Mean Absolute Error: 0.27
Adjusted R^2 Score: 0.88
MAPE:  119802.26286139587
MPE:  -119784.1991655506
time: 16min 5s (started: 2023-07-06 11:54:59 +03:00)


In [8]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y


# DT
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=100, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100building tree 40 of 100
building tree 41 of 100
building tree 42 of 100bui

[Parallel(n_jobs=-1)]: Done  56 out of 100 | elapsed: 38.3min remaining: 30.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 47.5min finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:   19.9s remaining:   15.7s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:   25.3s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:  1.3min remaining:  1.0min
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:   19.8s remaining:   15.6s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:   26.2s finished


R2 on training data: 0.91
R2 on validation data: 0.88
Mean Squared Error: 9.04
Mean Absolute Error: 0.28
Adjusted R^2 Score: 0.88
MAPE:  143533.2968189135
MPE:  -143515.78679002906
time: 50min 48s (started: 2023-07-06 12:11:04 +03:00)


# +density


In [9]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 13.9min remaining: 29.5min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 16.7min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 17.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    6.0s remaining:  1.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.7s remaining:    4.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    8.5s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   24.8s remaining:  4.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   26.4s remaining:   17.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   27.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.91
R2 on validation data: 0.89
Mean Squared Error: 8.35
Mean Absolute Error: 0.27
Adjusted R^2 Score: 0.89
MAPE:  119431.46528088521
MPE:  -119413.63077847898
time: 18min 41s (started: 2023-07-06 13:01:53 +03:00)


# +density


In [10]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 10.8min remaining: 22.9min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 13.7min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 13.9min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.8s remaining:   55.5s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    5.4s remaining:    3.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.9s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   19.6s remaining:  3.7min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   21.4s remaining:   14.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   23.2s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.91
R2 on validation data: 0.88
Mean Squared Error: 9.12
Mean Absolute Error: 0.28
Adjusted R^2 Score: 0.88
MAPE:  148118.49267444783
MPE:  -148101.14486232476
time: 15min 12s (started: 2023-07-06 13:20:34 +03:00)


# +density+gdpcap


In [11]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 19.0min remaining: 40.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 22.2min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 22.5min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.4s remaining:   50.6s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    5.2s remaining:    3.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    6.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   18.6s remaining:  3.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   20.7s remaining:   13.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   23.2s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.91
R2 on validation data: 0.89
Mean Squared Error: 8.4
Mean Absolute Error: 0.27
Adjusted R^2 Score: 0.89
MAPE:  117044.15491994303
MPE:  -117026.3868158657
time: 24min 6s (started: 2023-07-06 13:35:47 +03:00)


# +density+gdpcap


In [12]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 15.6min remaining: 33.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 20.6min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 21.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    5.0s remaining:   58.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    5.4s remaining:    3.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    6.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   20.2s remaining:  3.9min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   21.7s remaining:   14.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   25.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.89
Mean Squared Error: 8.35
Mean Absolute Error: 0.28
Adjusted R^2 Score: 0.89
MAPE:  141193.79912448706
MPE:  -141176.44921195152
time: 22min 38s (started: 2023-07-06 13:59:53 +03:00)


# +density+gdpcap+pci


In [13]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 20.0min remaining: 42.4min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 23.0min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 23.3min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.0s remaining:   46.2s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.4s remaining:    3.0s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.2s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   16.0s remaining:  3.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   17.6s remaining:   11.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   19.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.92
R2 on validation data: 0.9
Mean Squared Error: 7.55
Mean Absolute Error: 0.28
Adjusted R^2 Score: 0.9
MAPE:  122999.20123229867
MPE:  -122981.67849384114
time: 24min 46s (started: 2023-07-06 14:22:31 +03:00)


# +density+gdpcap+pci


In [14]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 14.8min remaining: 31.5min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 17.6min remaining:  3.4min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 17.8min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.1s remaining:   35.1s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.5s remaining:    2.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   12.5s remaining:  2.4min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   14.4s remaining:    9.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   16.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.9
R2 on validation data: 0.87
Mean Squared Error: 9.78
Mean Absolute Error: 0.31
Adjusted R^2 Score: 0.87
MAPE:  161651.71287696532
MPE:  -161634.80722462534
time: 18min 58s (started: 2023-07-06 14:47:18 +03:00)


# +density+gdpcap+pci + No year


In [15]:
data.columns

Index(['refYear', 'partnerCode', 'reporterCode', 'distw_harmonic', 'contig',
       'gdp_d', 'gdp_o', 'gdpcap_d', 'gdpcap_o', 'pop_d', 'pop_o', 'wto_d',
       'wto_o', 'refMonth', 'partner2Code', 'cmdCode', 'primaryValue',
       '2HScmdCode', '4HScmdCode', 'pci', 'density'],
      dtype='object')

time: 4.26 ms (started: 2023-07-06 15:06:16 +03:00)


In [16]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 14.1min remaining: 29.9min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 16.6min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 16.8min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.0s remaining:   35.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.5s remaining:    2.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   11.8s remaining:  2.3min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   13.6s remaining:    9.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   15.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.87
R2 on validation data: 0.85
Mean Squared Error: 10.77
Mean Absolute Error: 0.32
Adjusted R^2 Score: 0.85
MAPE:  144546.2932730032
MPE:  -144529.2456545511
time: 17min 55s (started: 2023-07-06 15:06:16 +03:00)


# +density+gdpcap+pci + No year


In [17]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 12.8min remaining: 27.2min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 15.3min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 15.4min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    2.3s remaining:   27.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    2.9s remaining:    1.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    9.6s remaining:  1.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   11.2s remaining:    7.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   12.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.86
R2 on validation data: 0.83
Mean Squared Error: 12.44
Mean Absolute Error: 0.35
Adjusted R^2 Score: 0.83
MAPE:  178303.11283933817
MPE:  -178286.752384887
time: 16min 29s (started: 2023-07-06 15:24:11 +03:00)


# Log transformations

In [18]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  8.5min remaining: 18.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 11.6min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 11.9min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    7.8s remaining:  1.5min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    8.2s remaining:    5.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    9.5s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   31.5s remaining:  6.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   33.3s remaining:   22.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   35.7s finished


R2 on training data: 0.26
R2 on validation data: 0.25
MSE:  55.67
MAE:  0.28
R-square:  0.25
MAPE:  6299.48
MPE:  -6226.03
time: 13min 57s (started: 2023-07-06 15:40:40 +03:00)


In [19]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25

building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 14.9min remaining: 31.7min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 17.0min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 17.8min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    9.5s remaining:  1.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   10.8s remaining:    7.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   12.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   35.9s remaining:  6.9min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   40.8s remaining:   27.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   47.6s finished


R2 on training data: 0.23
R2 on validation data: 0.22
MSE:  57.42
MAE:  0.29
R-square:  0.22
MAPE:  6116.01
MPE:  -6042.64
time: 20min 26s (started: 2023-07-06 15:54:38 +03:00)


In [20]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 21.4min remaining: 45.4min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 27.0min remaining:  5.1min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 28.3min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   10.3s remaining:  2.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   13.3s remaining:    8.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   15.9s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   34.4s remaining:  6.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   40.7s remaining:   27.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   48.2s finished


R2 on training data: 0.22
R2 on validation data: 0.21
MSE:  58.31
MAE:  0.29
R-square:  0.21
MAPE:  5884.83
MPE:  -5811.65
time: 32min 21s (started: 2023-07-06 16:15:05 +03:00)


In [21]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 28.9min remaining: 61.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 29.9min remaining:  5.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 36.9min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   20.8s remaining:  4.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   24.9s remaining:   16.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   26.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  1.4min remaining: 16.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:  1.8min remaining:  1.2min
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:  1.9min finished


R2 on training data: 0.24
R2 on validation data: 0.24
MSE:  56.13
MAE:  0.28
R-square:  0.24
MAPE:  5829.2
MPE:  -5755.97
time: 43min 5s (started: 2023-07-06 16:47:26 +03:00)


In [22]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 33.6min remaining: 71.4min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 35.5min remaining:  6.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 37.5min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   16.6s remaining:  3.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   24.2s remaining:   16.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   26.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  1.1min remaining: 13.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:  1.3min remaining:   52.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:  1.4min finished


R2 on training data: 0.28
R2 on validation data: 0.26
MSE:  54.57
MAE:  0.28
R-square:  0.26
MAPE:  6058.46
MPE:  -5985.04
time: 43min 58s (started: 2023-07-06 17:30:32 +03:00)
